# Quick start

*EPAM Syngen* is an unsupervised tabular data generation tool based on a variational autoencoder (VAE). 
It supports common tabular datatypes (floats, integers, datetime, text, categorical, binary) and can generate linked tables that sharing keys using the simple statistical approach. 
The SDK exposes simple programmatic entry points for training, inference, report generation, loading and saving data in supported formats - *CSV*, *Avro* and *Excel* format. The data should be located locally and be in UTF-8 encoding.

This notebook demonstrates the SDK usage. Install the package and then you can call the main SDK class `Syngen` to run training, inference or generation of reports, and the class`DataIO` to load and save the data in supported formats.

Python *3.10* or *3.11* is required to run the library. The library is tested on Linux and Windows operating systems.

# Installation

Please, install the library *syngen* (from Pypi):

In [1]:
!pip install  --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ --use-pep517 --no-cache-dir syngen==0.10.28rc1

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/


## Launch training

You can start training process using the SDK entrypoint `Syngen().train(...)`. This will train a model and save the model artifacts to a disk in the directory *'./model_artifacts'*. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
Syngen.train(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' format to train multiple tables or provide all parameters in a file
    table_name: Optional[str] = None,     # required for single-table training
    source: Optional[str] = None,         # path to original data
    epochs: int = 10,                     # number of training epochs
    drop_null: bool = False,              # whether to drop rows with at least one missing value
    row_limit: Optional[int] = None,      # a number of rows to train over
    reports: Union[str, List[str]] = "none",  # report types: 'none', 'accuracy', 'sample', 'metrics_only', 'all'
    log_level: str = "INFO",              # logging level
    batch_size: int = 32,                 # training batch size
    fernet_key: Optional[str] = None      # name of the environment variable containing the Fernet key for secure storage of the data subset
)
```

### Parameters description:

- **`metadata_path`** *(Optional[str], default: None)*: Path to a metadata file in the *'.yaml'* or *'.yml'* format for training multiple tables or specifying per-table settings. If provided, individual table parameters (`source`, `table_name`) are ignored in favor of metadata values.

- **`table_name`** *(Optional[str], default: None)*: **Required for single-table training**. An arbitrary name used to identify the table and name the artifact folders. Must be provided when not using `metadata_path`.

- **`source`** *(Optional[str], default: None)*: **Required for single-table training**. Path to the original data file.

- **`epochs`** *(int, default: 10)*: Number of training epochs. Must be ≥ 1. Since the early stopping mechanism is implemented the bigger value of epochs is the better.

- **`drop_null`** *(bool, default: False)*: Whether to drop rows containing at least one missing value before training. When `False`, missing values are handled during the training process.

- **`row_limit`** *(Optional[int], default: None)*: Maximum number of rows to use for training. If specified and less than the total rows, a random subset of the specified size will be selected. Useful for testing or working with large datasets.

- **`reports`** *(Union[str, List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing synthetic data (same size as original) with original dataset to estimate the quality of training process
  - `"sample"` - generates a sample report showing distribution comparisons between the original data and the subset of this data
  - `"metrics_only"` - outputs metrics to stdout
  - `"all"` - generates both accuracy and sample reports
  - List example: `["accuracy", "sample"]` to generate multiple report types
  - *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: Logging level for the training process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`batch_size`** *(int, default: 32)*: Training batch size. Must be ≥ 1. Splits training into batches to optimize memory usage. Smaller batches use less RAM but may increase training time.

- **`fernet_key`** *(Optional[str], default: None)*: Name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is encrypted on disk (stored in the `.dat` format). If not provided, data is stored unencrypted as `.pkl` format. **Important**: The same key must be used during inference and report generation to decrypt the data.

### Key notes:

- **Single table training**: Both `source` and `table_name` are required when `metadata_path` is not provided.
- **Multiple tables training**: Use `metadata_path` to define multiple tables with or without relationships.

*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [1]:
# The example of training the single table

from syngen.sdk import Syngen

Syngen().train(
    source="../examples/example-data/housing.csv", 
    table_name="housing", 
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=32, 
    reports="all"
)

2025-10-20 08:23:57.849171: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-20 08:23:57.849245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-20 08:23:58.004436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-20 08:24:04.075 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
20

32/32 [==============================] - 0s 978us/step


2025-10-20 08:24:19.549 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-20 08:24:19.549 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-20 08:24:21.649 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-20 08:24:21.707 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2025-10-20 08:24:21.708 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-20 08:24:21.708 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-20 08:24:21.708 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing' was completed
2025-10-20 08:24:21.710 | INFO     | syngen.ml.worker.worker:_infer_table:518 - Infer process of the table - 'housing' has started


 1/32 [..............................] - ETA: 4s

2025-10-20 08:24:22.155 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2025-10-20 08:24:22.156 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2025-10-20 08:24:22.156 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2025-10-20 08:24:22.156 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-20 08:24:22.156 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


32/32 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 9654.18it/s]
2025-10-20 08:24:22.423 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-20 08:24:22.428 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2025-10-20 08:24:22.428 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-20 08:24:22.428 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of sample metrics for the table - 'housing' has started
2025-10-20 08:24:25.569 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The sample report of the table - 'housing' has been generated
2025-10-20 08:24:25.569 | INFO     | syngen.ml.reporters.reporters:_log_and_update_pro

In [ ]:
# The example of training of multiple tables with relationships

from syngen.sdk import Syngen

Syngen().train(
    metadata_path="../examples/example-metadata/housing_metadata.yaml",
    log_level="DEBUG"
)

2025-10-16 11:15:53.891 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-16 11:15:53.897 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid
2025-10-16 11:15:53.899 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2025-10-16 11:15:53.900 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2025-10-16 11:15:53.900 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2025-10-16 11:15:53.906

25/25 [==============================] - 0s 835us/step


2025-10-16 11:16:06.892 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-16 11:16:06.893 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-16 11:16:07.446 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-16 11:16:07.479 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-16 11:16:07.479 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-16 11:16:07.479 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-16 11:16:07.479 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing_properties' was completed
2025-10-16 11:16:07.487 | INFO     | syngen.ml.processors.processors:_preprocess_data:120 - As the parameter 'drop_null' set t

57/57 [==============================] - 0s 777us/step


2025-10-16 11:16:20.528 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-16 11:16:20.528 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-16 11:16:21.805 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-16 11:16:21.833 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-conditions/vae/checkpoints
2025-10-16 11:16:21.834 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-16 11:16:21.834 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-16 11:16:21.834 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing_conditions' was completed
2025-10-16 11:16:21.836 | INFO     | syngen.ml.worker.worker:_infer_table:518 - Infer process of the table - 'housing_properti

...

2025-10-16 11:16:22.132 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-16 11:16:22.132 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=790, run_parallel=False, batch_size=790, random_seed=1
2025-10-16 11:16:22.132 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2025-10-16 11:16:22.132 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2025-10-16 11:16:22.132 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:16:22.133 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 7618.09it/s]
2025-10-16 11:16:22.305 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table 

32/32 [==============================] - 0s 785us/step


Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4139.12it/s]

25/25 [==============================] - 0s 788us/step



2025-10-16 11:16:22.623 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 2 of 2
2025-10-16 11:16:22.623 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:16:22.623 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4684.63it/s]
2025-10-16 11:16:22.703 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-16 11:16:22.720 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2025-10-16 11:16:22.721 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:1

# Launch inference

You can start inference process using the SDK entrypoint `Syngen().infer(...)`. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
Syngen().infer(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' to infer multiple tables or provide all params in a file
    table_name: Optional[str] = None,     # required for single-table inference
    size: int = 100,                      # the desired number of rows to generate
    run_parallel: bool = False,           # whether to use multiprocessing (feasible for tables > 5000 rows)
    batch_size: Optional[int] = None,     # inference batch size
    random_seed: Optional[int] = None,    # if specified, generates a reproducible result
    reports: Union[str, List[str]] = "none",  # report types: 'none', 'accuracy', 'metrics_only', 'all'
    log_level: str = "INFO",              # logging level
    fernet_key: Optional[str] = None      # name of the environment variable containing the Fernet key for decrypting the data subset
)
```

### Parameters description:

- **`metadata_path`** *(Optional[str], default: None)*: Path to a metadata file in the *'.yaml'* or *'.yml'* format for inferring multiple tables or specifying per-table settings. If provided, the individual table parameter `table_name` is ignored in favor of metadata values.

- **`table_name`** *(Optional[str], default: None)*: **Required for single-table inference**. The name of the table to generate synthetic data for. Must match the name used during training.

- **`size`** *(int, default: 100)*: The desired number of synthetic rows to generate. Must be ≥ 1.

- **`run_parallel`** *(bool, default: False)*: Whether to use multiprocessing for data generation. Set to `True` to enable parallel processing, which is recommended and feasible for generating large tables (>5000 rows).

- **`batch_size`** *(Optional[int], default: None)*: Inference batch size. Must be ≥ 1. If specified, the generation is split into batches to optimize memory usage and save RAM.

- **`random_seed`** *(Optional[int], default: None)*: Random seed for reproducible generation. Must be ≥ 0.

- **`reports`** *(Union[str, List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data patterns to verify quality of a generated data
  - `"metrics_only"` - outputs metrics to stdout without generating an accuracy report
  - `"all"` - generates an accuracy report (same as `"accuracy"`)
  - List example: `["accuracy", "metrics_only"]` to generate multiple report types
  - *Note*: Report generation may require significant time for large generated tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: Logging level for the inference process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`fernet_key`** *(Optional[str], default: None)*: Name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is decrypted for report generation. **Important**: The same key used during a training must be used during a report generation to successfully decrypt the data.

### Key notes:

- **Single table inference**: `table_name` is required when `metadata_path` is not provided, and it must match the name used during the training process.
- **Multiple tables inference**: Use `metadata_path` to generate multiple tables with or without relationships simultaneously.

*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [4]:
# The example of inference for the single table
from syngen.sdk import Syngen

Syngen.infer(
    table_name="housing", 
    size=200,
    run_parallel=False,
    batch_size=32, 
    random_seed=42,
    reports="all"
)

2025-10-16 11:21:10.638 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-16 11:21:10.640 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2025-10-16 11:21:10.640 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2025-10-16 11:21:10.640 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_a

1/1 [==============================] - 0s 123ms/step


2025-10-16 11:21:11.051 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2025-10-16 11:21:11.052 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 7 batch(es)
2025-10-16 11:21:11.052 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 7
2025-10-16 11:21:11.052 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:21:11.052 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 12520.31it/s]

1/1 [==============================] - 0s 13ms/step



2025-10-16 11:21:11.204 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:21:11.204 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 2 of 7
2025-10-16 11:21:11.204 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:21:11.204 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 16120.67it/s]
2025-10-16 11:21:11.245 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:21:11.246 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 3 of 7
2025-10-16 11:21:11.246 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2

...


2025-10-16 11:21:11.325 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:21:11.325 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 5 of 7
2025-10-16 11:21:11.325 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:21:11.325 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 15634.48it/s]

1/1 [==============================] - 0s 13ms/step



2025-10-16 11:21:11.365 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:21:11.365 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 6 of 7
2025-10-16 11:21:11.366 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:21:11.366 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 14632.84it/s]
2025-10-16 11:21:11.405 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 14632.84it/s]
2025-10-16 11:21:11.405 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 1

1/1 [==============================] - 0s 128ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 11040.28it/s]
2025-10-16 11:21:11.564 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 11040.28it/s]
2025-10-16 11:21:11.564 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:21:11.567 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2025-10-16 11:21:11.567 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:21:11.567 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16

In [5]:
# The example of inference of multiple tables with relationships

from syngen.sdk import Syngen

Syngen.infer(
    metadata_path="../examples/example-metadata/housing_metadata.yaml",
    log_level="DEBUG"
)

2025-10-16 11:21:36.590 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-16 11:21:36.596 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid
2025-10-16 11:21:36.596 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv' was removed
2025-10-16 11:21:36.596 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:106 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/infer_message.success' was removed
2025-10-16 11:21:36.596 | INFO     | syngen.ml.worker.worker:_r

3/3 [==============================] - 0s 2ms/step


2025-10-16 11:21:36.901 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2025-10-16 11:21:36.902 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=90, run_parallel=False, batch_size=90, random_seed=10, reports - 'accuracy'
2025-10-16 11:21:36.902 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2025-10-16 11:21:36.902 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2025-10-16 11:21:36.902 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:21:36.902 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 9579.16it/s]
2025-10-16 11:21:37.033 | INFO     | syngen.ml.strategies.strategies:run:242 - Sy

3/3 [==============================] - 0s 2ms/step


Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 5562.74it/s]
2025-10-16 11:21:37.330 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-16 11:21:37.334 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2025-10-16 11:21:37.334 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:21:37.334 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing_conditions' has started

2025-10-16 11:21:37.330 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2025-10-16 11:21:37.334 | INFO     | syngen.ml.strategies.strategies:run:242 - Synth

# Data security: Using Fernet Key for encryption

In the current implementation, a sample of the original data is stored on a disk during the training process. To ensure data security and protect sensitive information, you can use a **Fernet key** to encrypt this data.

## What is a Fernet Key?

A Fernet key is a 44-character URL-safe base64-encoded string used for symmetric encryption. When provided, the data subset is encrypted on a disk (stored in `.dat` format instead of unencrypted `.pkl` format).

## How to Generate a Fernet Key

You can generate a Fernet key using the following code:

In [6]:
# Generate a Fernet key
from cryptography.fernet import Fernet

fernet_key = Fernet.generate_key().decode("utf-8")

## Setting the Fernet Key as an environment variable

After generating the key, you need to store it as an environment variable. This can be done in your terminal or programmatically in Python.

### Option 1: Set in Terminal (Linux/macOS)

```bash
export MY_FERNET_KEY='your_generated_fernet_key_here'
```

### Option 2: Set in Terminal (Windows)

```cmd
set MY_FERNET_KEY=your_generated_fernet_key_here
```

### Option 3: Set programmatically in Python

```python
import os
os.environ['MY_FERNET_KEY'] = 'your_generated_fernet_key_here'
```

## Using the Fernet Key in a training

When training with encryption, pass the name of the environment variable (not the key itself) to the `fernet_key` parameter:

In [7]:
# The example: the training with the Fernet key encryption

import os
from syngen.sdk import Syngen

# Step 1: Set the Fernet key as an environment variable
os.environ['MY_FERNET_KEY'] = fernet_key  # Using the key generated above

# Step 2: Train with encryption enabled
Syngen.train(
    source="../examples/example-data/housing.csv", 
    table_name="housing_encrypted", 
    epochs=5,
    row_limit=1000, 
    batch_size=32, 
    fernet_key="MY_FERNET_KEY"  # Pass the environment variable name, not the key itself
)

2025-10-16 11:22:02.860 | INFO     | syngen.train:launch_train:69 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-16 11:22:02.864 | WARNING  | syngen.ml.config.validation:_launch_validation:394 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2025-10-16 11:22:02.865 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2025-10-16 11:22:02.865 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2025-10-16 11:22:02.865 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the track

32/32 [==============================] - 0s 864us/step


2025-10-16 11:22:16.902 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2025-10-16 11:22:16.902 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2025-10-16 11:22:18.736 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2025-10-16 11:22:18.775 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-encrypted/vae/checkpoints
2025-10-16 11:22:18.775 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2025-10-16 11:22:18.775 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2025-10-16 11:22:18.776 | INFO     | syngen.ml.strategies.strategies:run:160 - Training of the table - 'housing_encrypted' was completed
2025-10-16 11:22:18.778 | INFO     | syngen.ml.data_loaders.data_loaders:save_data:681 - Data is successfully encrypted and save

## Using the Fernet Key in an inference

**Important**: When generating synthetic data with inference, you must use the **same Fernet key** that was used during training. This allows the system to decrypt the stored data subset for report generation.

If the Fernet key is not provided or doesn't match the training key, the inference process will fail when trying to access the encrypted data.

In [8]:
# Example: Inference with Fernet key decryption

from syngen.sdk import Syngen

# The environment variable 'MY_FERNET_KEY' is already set from the training step

# Inference with the same Fernet key
Syngen.infer(
    table_name="housing_encrypted",  # Must match the name used in training
    size=200,
    batch_size=32, 
    random_seed=42,
    reports="all",
    fernet_key="MY_FERNET_KEY"  # Must use the same key as in training
)

2025-10-16 11:31:02.003 | INFO     | syngen.infer:launch_infer:43 - The inference process will be executed according to the information mentioned in 'infer_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted
2025-10-16 11:31:02.004 | WARNING  | syngen.ml.config.validation:_launch_validation:394 - Encryption and decryption are enabled for the table 'housing_encrypted' as a Fernet key is provided
2025-10-16 11:31:02.005 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing_encrypted', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2025-10-16 11:31:02.006 | INFO     | syngen.ml.data_loaders.data_loaders:load_data:706 - Data stored at the path - 'model_arti

1/1 [==============================] - 0s 124ms/step


2025-10-16 11:31:02.412 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-encrypted/vae/checkpoints
2025-10-16 11:31:02.412 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 7 batch(es)
2025-10-16 11:31:02.413 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 1 of 7
2025-10-16 11:31:02.413 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:31:02.413 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13731.35it/s]

1/1 [==============================] - 0s 15ms/step



2025-10-16 11:31:02.566 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.566 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 2 of 7
2025-10-16 11:31:02.566 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:31:02.567 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13490.45it/s]
2025-10-16 11:31:02.610 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.610 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 3 of 7
2025-10-16 11:31:02.611 | INFO     | syngen.ml.handlers.handlers:ru

...


2025-10-16 11:31:02.692 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.692 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 5 of 7
2025-10-16 11:31:02.692 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:31:02.692 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13092.32it/s]
2025-10-16 11:31:02.732 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.732 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 6 of 7
2025-10-16 11:31:02.732 | INFO     | syngen.ml.handlers.handlers:ru

1/1 [==============================] - 0s 13ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 13850.90it/s]
2025-10-16 11:31:02.771 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.772 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 7 of 7
2025-10-16 11:31:02.772 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2025-10-16 11:31:02.772 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_encrypted' started.

2025-10-16 11:31:02.771 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.772 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_encrypted'. Generating the batch 7 of 7
2025-10-16 11:31:02.772 | INFO     | syngen.ml.handlers.handlers:ru

1/1 [==============================] - 0s 128ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 11117.43it/s]
2025-10-16 11:31:02.932 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 11117.43it/s]
2025-10-16 11:31:02.932 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2025-10-16 11:31:02.935 | INFO     | syngen.ml.strategies.strategies:run:242 - Synthesis of the table - 'housing_encrypted' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-encrypted/merged_infer_housing-encrypted.csv'
2025-10-16 11:31:02.935 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:31:02.935 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'h

## Using the Fernet Key with the metadata file

You can also specify the Fernet key in the metadata file for both training and inference:

```yaml
global:
  encryption:
    fernet_key: MY_FERNET_KEY  # Name of the environment variable

TABLE_NAME:
  train_settings:
    source: "./data/table.csv"
  
  infer_settings:
    size: 100
  
  # You can also specify per-table encryption
  encryption:
    fernet_key: MY_FERNET_KEY
```

Then use it in your code:

```python
# Training with the metadata file and the Fernet key
Syngen.train(metadata_path="path/to/metadata.yaml")

# Inference with the metadata file and the Fernet key
Syngen.infer(metadata_path="path/to/metadata.yaml")
```

## Important security notes

⚠️ **Critical security considerations:**

1. **Store the key securely**: Never hardcode the Fernet key directly in your code or commit it to version control systems.
2. **Key recovery is impossible**: If you lose the Fernet key, encrypted data cannot be recovered
3. **Same key required**: Always use the same Fernet key for training and inference
4. **Environment variables**: Use environment variables to store the key securely
5. **Key length**: The Fernet key must be exactly 44 characters (URL-safe base64-encoded)
6. **Production environments**: In production, use secure secret management services (AWS Secrets Manager, Azure Key Vault, HashiCorp Vault, etc.)

## What happens without a Fernet key?

If you don't provide a `fernet_key` parameter:
- Data subset is stored **unencrypted** in `.pkl` format
- No decryption is needed during inference
- Suitable for non-sensitive data or development environments

With a `fernet_key`:
- Data subset is stored **encrypted** in `.dat` format
- Decryption is required during inference using the same key
- Recommended for sensitive or production data

# Generate reports separately

Sometimes you may want to generate reports separately after training or/and inference has already been completed. The SDK provides the `Syngen().generate_reports(...)` method that allows you to generate quality reports for a table using existing artifacts without re-running the training or/and inference processes.

This method is useful when:
- You completed training/inference without reports (with `reports="none"`)
- You want to generate additional report types later
- You want to separate the computation-intensive training/inference from report generation


```python
Syngen().generate_reports(
    table_name: str,                        # required: table name used in training/inference
    reports: Union[str, List[str]],         # required: report types to generate
    fernet_key: Optional[str] = None        # optional: Fernet key for decrypting encrypted data
)
```

### Parameters description:

- **`table_name`** *(str, required)*: The name of the table to generate reports for. Must match exactly the name used during the training or inference process.

- **`reports`** *(Union[str, List[str]], required)*: Controls which quality reports to generate. Accepts single string or list of strings:
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data
  - `"metrics_only"` - outputs metrics information to stdout without generating an accuracy report
  - `"sample"` - generates a sample report showing distribution comparisons between original data and the data subset used for a training process
  - `"all"` - generates all available reports (*"accuracy"* and *"sample"*)
  
  List example: `["accuracy", "sample"]` to generate multiple report types.

  *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`fernet_key`** *(Optional[str], default: None)*: The nae of the environment variable containing the Fernet key used to decrypt the original data subset. **Important**: Must be the same key used during training if the data was encrypted.

### Required artifacts

To generate reports, the following artifacts must exist:

**For accuracy reports (`"accuracy"` or `"metrics_only"`):**
- Training must be completed successfully
- Inference must be completed successfully

**For sample reports (`"sample"`):**
- Training must be completed successfully

### Key notes:

- The method uses existing artifacts and does not re-run training or inference
- All required artifacts must be present in the `model_artifacts` directory
- The `table_name` must match exactly what was used in training/inference
- If data was encrypted during training, the same `fernet_key` must be provided

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [9]:
# Example 1: Generate an accuracy report after training and inference completed without reports

from syngen.sdk import Syngen

# Assume training and inference were already completed with reports="none"
# Now generate an accuracy report separately
Syngen().generate_reports(
    table_name="housing",
    reports="accuracy"
)

2025-10-16 11:31:54.037 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:31:54.038 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16 11:31:54.390 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
2025-10-16 11:31:54.390 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
Generating bivariate distributions...: 100%|██████████| 45/45 [00:14<00:00,  3.07it/s]

2025-10-16 11:32:11.399 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:32:11.399 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:32:12.142 | INFO     | s

In [10]:
# Example 2: Generate a sample report after training completed

from syngen.sdk import Syngen

# Generate a sample report to compare original data with its subset
Syngen().generate_reports(
    table_name="housing",
    reports="sample"
)

2025-10-16 11:32:19.216 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:32:19.216 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16 11:32:19.571 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
2025-10-16 11:32:19.571 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
Generating bivariate distributions...: 100%|██████████| 45/45 [00:14<00:00,  3.03it/s]

2025-10-16 11:32:36.493 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:32:36.493 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:32:37.292 | INFO     | s

In [11]:
# Example 3: Generate multiple reports at once

from syngen.sdk import Syngen

# Generate both accuracy and sample reports
Syngen().generate_reports(
    table_name="housing",
    reports=["accuracy", "sample"]
)

# Or use "all" to generate all available reports
Syngen().generate_reports(
    table_name="housing",
    reports="all"
)

2025-10-16 11:32:46.888 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:32:46.888 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16 11:32:47.228 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
2025-10-16 11:32:47.228 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
Generating bivariate distributions...: 100%|██████████| 45/45 [00:14<00:00,  3.14it/s]

2025-10-16 11:33:04.003 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:33:04.003 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:33:04.791 | INFO     | s

In [12]:
# Example 4: Generate reports for encrypted data

import os
from syngen.sdk import Syngen

# Ensure the Fernet key environment variable is set
# (Should be the same key used during training)
os.environ['MY_FERNET_KEY'] = fernet_key

# Generate reports with decryption
Syngen().generate_reports(
    table_name="housing_encrypted",
    reports="accuracy",
    fernet_key="MY_FERNET_KEY"  # Same key used in training
)

2025-10-16 11:34:39.748 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:34:39.748 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16 11:34:40.103 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
2025-10-16 11:34:40.103 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
Generating bivariate distributions...: 100%|██████████| 45/45 [00:14<00:00,  3.11it/s]

2025-10-16 11:34:56.544 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:34:56.544 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:34:57.140 | INFO     | s

In [13]:
# Example 5: Generate metrics only (without a full accuracy report)

from syngen.sdk import Syngen

# Output metrics to stdout without generating HTML/PDF reports
Syngen().generate_reports(
    table_name="housing",
    reports="metrics_only"
)

2025-10-16 11:36:06.687 | WARNING  | syngen.ml.reporters.reporters:generate_report:262 - The report(s) generation might be time-consuming
2025-10-16 11:36:06.687 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:308 - The calculation of accuracy metrics for the table - 'housing' has started
2025-10-16 11:36:07.048 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
2025-10-16 11:36:07.048 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:169 - Median accuracy is 0.8196
Generating bivariate distributions...: 100%|██████████| 45/45 [00:15<00:00,  2.97it/s]

2025-10-16 11:36:24.286 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:36:24.286 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:197 - Median of differences of correlations is 0.1735
2025-10-16 11:36:25.262 | INFO     | s

# Data loading and saving: DataIO class

The SDK provides the `DataIO` class for loading and saving data in various supported formats with optional encryption and format settings. This class is useful when you need to:
- Load and save data in different file formats (CSV, Avro, Excel, etc.)
- Load and save data with specific format settings
- Work with encrypted data files

## Class initialization

```python
DataIO(
    path: str,                          # required: path to the data file
    fernet_key: Optional[str] = None,   # optional: Fernet key for encrypted data
    **kwargs                            # optional: format settings for CSV or Excel tables, or schema for AVRO file
)
```

### Parameters description:

- **`path`** *(str, required)*: the path to the data file to load or save. Supported formats include:
  - CSV files: `.csv`, `.psv`, `.tsv`, `.txt`
  - Avro files: `.avro`
  - Excel files: `.xls`, `.xlsx`

- **`fernet_key`** *(Optional[str], default: None)*: the name of the environment variable containing the Fernet key for encrypted data operations.

- **`**kwargs`**: Optional format settings or/and a schema for reading and writing data. Available parameters depend on the file format:

  **For tables in '.csv' format:**
  - `sep` *(str)*: Delimiter to use (e.g., `','`, `';'`, `'\t'`)
  - `quotechar` *(str)*: Character used to denote the start and end of a quoted item (default: `'"'`)
  - `quoting` *(str)*: Quoting behavior - `"all"`, `"minimal"`, `"non-numeric"`, `"none"`
  - `escapechar` *(str)*: Character used to escape other characters
  - `encoding` *(str)*: Encoding to use (e.g., `'utf-8'`, `'latin-1'`)
  - `header` *(Optional[int, List[int], Literal["infer"]])*: Row number(s) containing column labels and marking the start of the data
  - `skiprows` *(Optional[int, List[int]])*: Lines to skip at the start of the file
  - `on_bad_lines` *(Literal["error", "warn", "skip"])*: Action on bad lines - `"error"`, `"warn"`, `"skip"`
  - `engine` *(Optional[Literal["c", "python"]])*: Parser engine - `"c"`, `"python"`
  - `na_values` *(Opional[List[str]])*: Additional strings to recognize as NA/NaN

  **For Excel formats (.xls, .xlsx):**
  - `sheet_name` *(Optional[str, int, List[Union[int, str]])*: Name or index of the sheet to read

## Available methods

`load_data(**kwargs)`

Loads data from the specified file path and returns it as a pandas DataFrame.

`load_schema()`

Returns the original schema of the loaded data, including column names and data types. Available only for data in the '.avro' format.

`save_data(df, **kwargs)`

Saves a pandas DataFrame to the specified file path with(without) the configured format settings or schema.

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [5]:
# The example 1: Load CSV data with default settings

from syngen.sdk import DataIO

data_io = DataIO(path="../examples/example-data/housing.csv")

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

2025-10-16 13:11:33.672 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid


Loaded 20640 rows and 10 columns

First few rows:
   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  


In [6]:
# The example 2: Load CSV data with custom format settings

from syngen.sdk import DataIO

data_io = DataIO(
    path="../examples/example-data/escaped_quoted_table.csv",
    sep=',',           # delimiter
    quotechar='"',     # quote character
    quoting="minimal", # quoting style
    encoding='utf-8',  # encoding
    header=0           # use first row as header
)

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

2025-10-16 13:11:43.496 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid


Loaded 15 rows and 3 columns

First few rows:
    Name   Age                             Description
0   John    25          "John is a software engineer."
1   Jane    30   "Jane is a \"full-stack\" developer."
2   Mary    27             "Mary is a data scientist."
3    Bob    35      "Bob is a \"jack-of-all-trades\"."
4  Alice    28   "Alice is a \"front-end\" developer."


In [7]:
# The example 3: Load data and get schema information

from syngen.sdk import DataIO


data_io = DataIO(path="../examples/example-data/avro_file.avro")

df = data_io.load_data()

schema = data_io.load_schema()

print("Data schema:")
print(schema)

2025-10-16 13:13:13.166 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid


Data schema:
{'type': 'record', 'name': 'Root', 'fields': [{'name': 'gender', 'type': ['null', 'long']}, {'name': 'height', 'type': ['null', 'double']}, {'name': 'id', 'type': ['null', 'long']}]}


In [8]:
# The example 4: Save data to a file

import pandas as pd
from syngen.sdk import DataIO


sample_data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 30, 35, 40, 45],
    'city': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']
})


data_io = DataIO(
    path="../examples/example-data/sample_output.csv",
    sep=',',
    encoding='utf-8'
)


data_io.save_data(sample_data)

print("Data saved successfully to '../examples/example-data/sample_output.csv'")

2025-10-16 13:14:11.258 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:300 - The schema of the metadata is valid


Data saved successfully to '../examples/example-data/sample_output.csv'
